In [13]:
from pathlib import Path
import sys

import yaml

# project setup
project_root = Path("..").resolve().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

In [14]:
from research.src.models.training import DetectorTrainerConfig
from research.src.training.detector import run_detector_training

CONFIG_DIR = Path("../../configs/").resolve()
cfg_path = CONFIG_DIR / "detector_train.yaml"
with open(cfg_path) as f:
    raw_conf = yaml.safe_load(f)
config = DetectorTrainerConfig(**raw_conf)
config

DetectorTrainerConfig(base_model='rtdetr-l.pt', project_name='pepper_thesis_detector_test', dataset_yaml=PosixPath('../../data/out/distilled/data.yaml'), epochs=5, imgsz=640, batch_size=4, device='0', optimizer='auto', lr0=0.0001)

In [15]:
results = run_detector_training(cfg_path)

Ultralytics 8.4.7 🚀 Python-3.12.3 torch-2.9.1+cu130 CUDA:0 (NVIDIA GeForce RTX 5050 Laptop GPU, 7706MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../../data/out/distilled/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=rtdetr-l.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=run, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, per

[research.src.training.detector|WARNING]ONNX Export failed (You can still use .pt): No module named 'onnx'


In [32]:
results.confusion_matrix.print()

0.0 2.0 0.0 1.0 3.0 321.0
0.0 0.0 0.0 0.0 0.0 9.0
0.0 0.0 0.0 0.0 0.0 545.0
0.0 0.0 0.0 1.0 0.0 16.0
0.0 0.0 0.0 0.0 0.0 2.0
2.0 1.0 2.0 0.0 2.0 0.0
